# Content:
## Formatting Data
## Models
## Results

---

# Formatting Data:

In [ ]:
# Box 0
# All imports

import numpy as np
import pandas as pd
import datetime
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Box 1
# Load the data as is

data_frame = pd.read_csv('ks-projects-201801.csv')
data_frame.head()

In [ ]:
# Box 2
# Format the data to certain columns

labels = data_frame.values[:, 9]
features = data_frame.values[:, :]
#print(features[0])

#Clean Up
#OLD COLUMN ORDER: category, main_category, deadline, launched, usd_pledged_real, usd_gaol_real
features = np.delete(features, [0,1,4,6,8,9,11,12], 1) #remove certain rows

#swaps deadline and launch so launch is listed first
def Swap(arr, start_index, last_index):
    arr[:, [start_index, last_index]] = arr[:, [last_index, start_index]]

#NEW COLUMN ORDER: category, main_category, launched, deadline, usd_pledged_real, usd_gaol_real
Swap(features, 2, 3)


#the states we don't want
unwantedLabels = np.array(['canceled', 'live', 'suspended', 'undefined'])

#row index of the with states we don't want
rowIndexToDelete = np.zeros(labels.shape)
for i  in range(len(unwantedLabels)):
  rowIndexToDelete += labels == unwantedLabels[i]

#convert to bool array
rowIndexToDelete = rowIndexToDelete.astype(bool)


#delete those rows
labels = np.delete(labels, rowIndexToDelete, axis=0)
features = np.delete(features, rowIndexToDelete, axis=0)

print('array of labels: shape ' + str(np.shape(labels)))
print('array of feature matrix: shape ' + str(np.shape(features)))

# np.split(data_frame, [9], axis=1)[1].head()

In [ ]:
print(features[0])
x = datetime.datetime.strptime(features[0][2], "%Y-%m-%d %H:%M:%S")
y = datetime.datetime.strptime(features[0][3], "%Y-%m-%d")

print( (y-x).days)

In [ ]:
#box 3

#***********************************
#Formating data to read time legibly
#***********************************

#COLUMN ORDER: category, main_category, launched, deadline, backers, usd_pledged_real, usd_gaol_real


for i in range(len(features)):
  #convert launched(string) to time in seconds
  features[i][2] = datetime.datetime.strptime(features[i][2], "%Y-%m-%d %H:%M:%S")
  #convert deadline(string) to time in seconds
  features[i][3] = datetime.datetime.strptime(features[i][3], "%Y-%m-%d")

#print(features[0][2])
#print(features[0][3])

In [ ]:
#box 4

#******************************************************
#Hot One Encoding for category and Main category columns
#******************************************************


#COLUMN ORDER: sub_category, main_category, launched, deadline, backers, usd_pledged_real, usd_gaol_real

MAIN_CAT_SIZE =  np.unique(features[:,1]).size
SUB_CAT_SIZE = np.unique(features[:,0]).size

#new shape should be (331675, 179)
#Hot One Encoding for both sub_cat and main_cat FOR FEATURES
nf = np.zeros((len(features), 179), dtype=object)
print(nf.shape)
subCatUnique = np.unique(features[:,0])
mainCatUnique = np.unique(features[:,1])
for i in range(len(features)):
  subMatch = (subCatUnique == features[i][0]).astype(int)
  mainMatch = (mainCatUnique == features[i][1]).astype(int)
  x = np.insert(features[i],1,subMatch)
  y = np.insert(x,-5, mainMatch)
  y = y[1:]
  #print(y)
  nf[i] = np.append(y[:SUB_CAT_SIZE],y[SUB_CAT_SIZE+1:])

 

In [ ]:
#box 5

#***************************
#Hot one encoding for labels
#***************************

uniqueLabels = np.unique(labels)
print(uniqueLabels)

mask = labels == uniqueLabels[1]

labels = mask.astype(int)


#0 == failed
#1 == successful

In [ ]:
#box 6

#*********************************
#Maintains an old copy of features
#*********************************

oldFeatures = features;
features = nf

In [ ]:
#box 7 

#*************************************************************************
#Replace Launched and Deadline with Duration (Added to the end of the row)
#*************************************************************************

nf = np.zeros( (len(features), 178), dtype=object)
print(nf.shape)
for i in range(len(features)):
    launched = features[i][-5]
    deadline = features[i][-4]
    duration = (deadline - launched).days
    x = np.copy(features[i])
    x = np.delete(x, [-4, -5])
    #print(x)
    nf[i] = np.append(x, duration)

#new shape should be (331675, 178)

In [ ]:
#box 6

#*****************************************************
#Maintains an old copy of features with datetime in it
#*****************************************************
oldFeaturesTime = features
features = nf


In [ ]:
#box 7

#***********************************************************
#Generates %80 training data set and %20 validation data set
#***********************************************************

np.random.seed(seed=1)
def IDK (features, labels):
  #20% of the data
  validAmount = len(features) * 0.2

  #list of all possible indicies inisde features
  indices = np.arange(0, len(features))

  #random choose indicies (20% of features)
  validIndicies = np.random.choice(indices, size=int(validAmount), replace=False)
  validIndicies = np.sort(validIndicies)

  #list of all possible indicies inisde features
  indices = np.arange(0, len(features))

  #random choose indicies (20% of features)
  validIndicies = np.random.choice(indices, size=int(validAmount), replace=False)

  #The Validation Set
  validSet = np.empty((int(validAmount), len(features[0])))
  validLabels = np.empty((int(validAmount)))

  #add the respective indicies for the validation set
  for i in range(len(validSet)):
      validSet[i] = np.copy(features[validIndicies[i]])
      validLabels[i] = np.copy(labels[validIndicies[i]])

  #remove those indicies from features
  trainingSet = np.delete(features, validIndicies, axis=0)
  trainingLabels = np.delete(labels, validIndicies);
  return (validSet, validLabels, trainingSet, trainingLabels)

validSet, validLabels, trainingSet, trainingLabels = IDK(features, labels)

print("validSet:")
print(validSet.shape)
print("validLabels:")
print(validLabels.shape)
print("trainingSet:")
print(trainingSet.shape)
print("trainingLabels:")
print(trainingLabels.shape);

print("features:")
print(features.shape)
print("labels:")
print(labels.shape);


In [ ]:
#7.5 testing
test = features #remove launched and dealine

TvalidSet, TvalidLabels, TtrainingSet, TtrainingLabels = IDK(test, labels)



In [ ]:
#box 8
print(test[5].shape)
#print(TtrainingSet)
#print(TvalidSet)
#nrows, ncols = TtrainingSet.shape
#dtype = {'names': ['f{}'.format(i) for i in range(ncols)], 'formats': ncols * [TtrainingSet.dtype]}
#duplicate = np.intersect1d(TtrainingSet.view(dtype), TvalidSet.view(dtype)).view(TtrainingSet.dtype).reshape(-1, TtrainingSet.shape[1])
#print(duplicate.shape)
knn = KNeighborsClassifier(n_neighbors=10, weights='uniform', p=2)
knn.fit(TtrainingSet, TtrainingLabels)
print(f"{knn.predict(TvalidSet[0:10])} vs {TvalidLabels[0:10]}")
knn.score(TvalidSet, TvalidLabels)

# Models
## insert all models here

### Feauture Selection:
### PCA:


# Results